In [ ]:
import os
import datetime
import h5py
import numpy as np
import imageio
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from databroker import Broker
from eiger_io.fs_handler import EigerHandler

In [ ]:
db = Broker.named('chx')
db.reg.register_handler('AD_EIGER2', EigerHandler, overwrite=True)

In [ ]:
def save_hdf5(data, filename='data.h5', dataset='dataset', save_dir=None):
    """Saving data to an hdf5 file.
        
        Parameters
        ----------
        data: ndarray
            data array to save into hdf5 file
        filename: str
            the name of the file
        dataset: str
            the name of the entry in the hdf5 layout
        save_dir: None or str
            a directory to save the file in
        
        Returns
        -------
        status: str
            a status string
    """
    if save_dir:
        filename = os.path.join(save_dir, filename)
    h5f = h5py.File(filename, 'w')
    r = h5f.create_dataset(dataset, data=data)
    status = '{} created: {}'.format(r, os.path.abspath(filename))
    h5f.close()
    return status

# Get data from Eiger detector

In [ ]:
eiger_uid = 'f4fe7c'
hdr_eiger = db[eiger_uid]  # 'eiger4m_single_image' field for hdr.data()
time_e = datetime.datetime.fromtimestamp(timestamp=hdr_eiger['start']['time']).strftime('%Y%m%d%H%M')
uid_e = hdr_eiger['start']['uid']
scan_id_e = hdr_eiger['start']['scan_id']
print(f'time: {time_e}\nuid: {uid_e}\nscan_id: {scan_id_e}')

In [ ]:
hdr_eiger.table()

In [ ]:
hdr_eiger.fields()

### Use the field with an '_image' occurrence

In [ ]:
imgs_eiger = list(hdr_eiger.data('eiger4m_single_image'))

In [ ]:
imgs_eiger[0].shape

In [ ]:
%matplotlib widget
plt.imshow(imgs_eiger[0][0], cmap='viridis', norm=LogNorm())

# Get data from xray eye camera

In [ ]:
xeye_uid = '44aae6fe'
hdr_xeye = db[xeye_uid]  # 'xray_eye3_image' field for hdr.data
time_x = datetime.datetime.fromtimestamp(timestamp=hdr_xeye['start']['time']).strftime('%Y%m%d%H%M')
uid_x = hdr_xeye['start']['uid']
scan_id_x = hdr_xeye['start']['scan_id']
print(f'time: {time_x}\nuid: {uid_x}\nscan_id: {scan_id_x}')

In [ ]:
hdr_xeye.table()

In [ ]:
hdr_xeye.fields()

### Use the field with an '_image' occurrence

In [ ]:
imgs_xeye = list(hdr_xeye.data('xray_eye3_image'))

In [ ]:
imgs_xeye[0].shape

In [ ]:
%matplotlib widget
plt.imshow(np.mean(imgs_xeye[0], axis=0), cmap='gray')

# Get tabular data

In [ ]:
hdr_tab = db['635f98ba']

In [ ]:
tbl = hdr_tab.table()
tbl

In [ ]:
tbl.plot(x='smp_y', y='mca_xs_channel1_rois_roi01_value_sum', grid=True)

In [ ]:
tbl.to_csv('/home/mrakitin/work/CHX/exported.csv')

# Save Eiger data to TIFF and HDF5 files

In [ ]:
filename = '{}_{}_{}'.format(time_e, eiger_uid, scan_id_e)
filename_tiff= '{}.{}'.format(filename, 'tiff')
filename_h5 = '{}.{}'.format(filename, 'h5')
print(filename_tiff)
print(filename_h5)

In [ ]:
imageio.imwrite(os.path.join('/home/mrakitin/work/CHX/', filename_tiff), imgs_eiger[0][0])

In [ ]:
status = save_hdf5(imgs_eiger, filename=filename_h5, dataset='dataset',
                   save_dir='/home/mrakitin/work/CHX/')
print(status)